In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import warnings
import tensorflow
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import keras_tuner as kt
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn import metrics

tensorflow.random.set_seed(20)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings("ignore")


## Split dataset

In [ ]:
# Read the CSV file containing filenames and labels
labels = pd.DataFrame(columns=['name'] + ['y' + str(i) for i in range(200)])
for file in tqdm(os.listdir('./curves')):
    tmp = pd.read_excel(f'./curves/{file}', index_col=0).iloc[1:,].T
    tmp = tmp.reset_index()
    tmp.columns = ['name'] + ['y' + str(i) for i in range(200)]
    labels = pd.concat([labels, tmp], axis=0)
labels['name'] = labels['name'] + '.jpg'


In [ ]:
labels2 = pd.read_excel('images_index.xlsx')
labels = pd.merge(labels2, labels, how='inner', on='name')


In [ ]:
labels

In [ ]:
# train_csv=labels[labels.subset=='train'][['filename']+['y'+str(i) for i in range(200)]]
# test_csv=labels[labels.subset=='test'][['filename']+['y'+str(i) for i in range(200)]]

In [ ]:
# Normalization
import joblib
ss = StandardScaler()
labels2 = pd.DataFrame(ss.fit_transform(labels[['y' + str(i) for i in range(200)]]), columns=['y' + str(i) for i in range(200)])
for col in ['y' + str(i) for i in range(200)]:
    labels[col] = labels2[col]
joblib.dump(ss,'ss.pkl')

In [ ]:
# Split training and testing sets
train_csv, test_csv = train_test_split(labels, test_size=0.3, random_state=1)


## Display the first 10

In [ ]:
# Visualization section
plt.figure(figsize=(12, 4), dpi=120)
for i in range(1, 11):
    y_values = labels.iloc[i-1, 3:203].values  # Change to 4:204 to ensure obtaining 200 y-values
    sns.lineplot(x=range(200), y=y_values)
plt.show()


## Read dataset

In [ ]:
# Custom data generator
class CustomDataGenerator(Sequence):
    def __init__(self, csv_file, directory, batch_size, target_size, label_list, shuffle=True):
        self.csv_file = csv_file
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.label_list = label_list
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.csv_file) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.csv_file.iloc[k] for k in indexes]
        X, y = self.__data_generation(batch)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.csv_file))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch):
        X1 = np.empty((self.batch_size, *self.target_size, 3))
        X2 = np.empty((self.batch_size, 1))
        X3 = np.empty((self.batch_size, 1))
        X4 = np.empty((self.batch_size, 1)) 
        y = np.empty((self.batch_size, len(self.label_list) - 3))

        for i, data in enumerate(batch):
            img_path = f"{self.directory}/{data['name']}"
            image = load_img(img_path, target_size=self.target_size)
            X1[i,] = img_to_array(image) / 255.0
            X2[i,] = data[self.label_list[0]]  # porosity (p)
            X3[i,] = data[self.label_list[1]]  # Lmin
            X4[i,] = data[self.label_list[2]]  # r
            y[i,] = data[self.label_list[3:]]  # curves

        return [X1, X2, X3, X4], y


In [ ]:
# Set the label list, including three auxiliary indicators
label_list = ['p', 'lmin', 'r'] + ['y' + str(i) for i in range(200)]
batch_size = 16
target_size = (256, 256)


In [ ]:
# Create training and testing data generators
train_generator = CustomDataGenerator(train_csv, './images', batch_size, target_size, label_list, shuffle=False)
test_generator = CustomDataGenerator(test_csv, './images', batch_size, target_size, label_list, shuffle=False)


In [ ]:
# Create a folder to save images
os.makedirs('train_images', exist_ok=True)
os.makedirs('test_images', exist_ok=True)


In [ ]:
# Save training set images
for i in tqdm(range(len(train_generator))):
    batch_data, batch_labels = train_generator[i]
    for j, img_array in enumerate(batch_data[0]):
        img_name = train_csv.iloc[i * batch_size + j]['name']
        img = array_to_img(img_array)
        img.save(os.path.join('train_images', img_name))

for i in tqdm(range(len(test_generator))):
    batch_data, batch_labels = test_generator[i]
    for j, img_array in enumerate(batch_data[0]):
        img_name = test_csv.iloc[i * batch_size + j]['name']
        img = array_to_img(img_array)
        img.save(os.path.join('test_images', img_name))


In [ ]:
# Check if GPU is enabled
import tensorflow as tf
tf.test.gpu_device_name()  
physical_devices = tf.config.experimental.list_physical_devices('GPU')  
for device in physical_devices:
     print(device)

In [ ]:
# Extract label data from the training and testing sets
y_train = []
for i in tqdm(range(len(train_generator))): 
    batch_data, batch_labels = train_generator[i]
    y_train.append(batch_labels)
y_train = np.concatenate(y_train)

y_test = []
for i in tqdm(range(len(test_generator))):
    batch_data, batch_labels = test_generator[i]
    y_test.append(batch_labels)
y_test = np.concatenate(y_test)
y_train.shape, y_test.shape


In [ ]:
y_train.shape,y_test.shape


In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(10,3),dpi=120)
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['mse'])+1),model_history.history['mse'])
    axs[0].plot(range(1,len(model_history.history['val_mse'])+1),model_history.history['val_mse'])
    axs[0].set_title('Model mse')
    axs[0].set_ylabel('mse')
    axs[0].set_xlabel('Epoch')
    # axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    # axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    # axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('Model iteration chart.jpg',dpi=600, bbox_inches = 'tight')
    plt.show()


## Automatic hyperparameter tuning

In [ ]:
# Modify the model-building function to support two auxiliary inputs
def build_model(hp):
    input_image = Input(shape=(256, 256, 3))
    input_features1 = Input(shape=(1,))  # p
    input_features2 = Input(shape=(1,))  # Lmin
    input_features3 = Input(shape=(1,))  # r

    base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    x = base_model(input_image)
    x = Flatten()(x)
    x = Concatenate()([x, input_features1, input_features2, input_features3])
    
    for i in range(hp.Int('num_layers', 2, 2)):
        x = Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=32), 
                  activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh']))(x)
        x = Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)

    output = Dense(200)(x)
    model = Model(inputs=[input_image, input_features1, input_features2, input_features3], outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
        loss='mse',
        metrics=['mse']
    )
    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=100,  
    factor=3,
    directory='my_dir',
    project_name='helloworld'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_generator, epochs=15, validation_data=test_generator, callbacks=[stop_early])
# Obtain the optimal hyperparameter combination
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 

# Print the optimal hyperparameters
print(f"""
The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')} 
with the following configuration:
""")
for i in range(best_hps.get('num_layers')):
    print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units, {best_hps.get(f'activation_{i}')} activation, {best_hps.get(f'dropout_{i}')} dropout")

print(f"Optimal learning rate: {best_hps.get('learning_rate')}")


In [ ]:
# def build_model(hp):
#     input_image = Input(shape=(256, 256, 3))
#     input_features = Input(shape=(1,))
#     base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
#     x = base_model(input_image)
#     x = Flatten()(x)
#     x = Concatenate()([x, input_features])
    
#     for i in range(hp.Int('num_layers', 2, 3)):
#         x = Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=32), 
#                   activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh']))(x)

#     output = Dense(200)(x)
#     model = Model(inputs=[input_image, input_features], outputs=output)
#     model.compile(
#         optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
#         loss='mse',
#         metrics=['mse']
#     )
#     return model

# tuner = kt.Hyperband(
#     build_model,
#     objective='val_loss',
#     max_epochs=3,  
#     factor=3,
#     directory='my_dir',
#     project_name='helloworld'
# )

# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# tuner.search(train_generator, epochs=15, validation_data=test_generator, callbacks=[stop_early])  

# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')} 
# with the following configuration:
# """)
# for i in range(best_hps.get('num_layers')):
#     print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units, {best_hps.get(f'activation_{i}')} activation")

# print(f"Optimal learning rate: {best_hps.get('learning_rate')}")

In [ ]:
# Build and train the best model
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[tf.keras.callbacks.ModelCheckpoint("./best_model.h5", save_best_only=True, monitor='val_loss')]
)


In [ ]:
plot_model_history(history)


In [ ]:
# Export loss values to a CSV file:
loss_values = history.history['loss'] 
val_loss_values = history.history['val_loss'] 

num_epoch = len(loss_values)

df_loss = pd.DataFrame({
    'epoch': np.arange(1, num_epoch + 1),
    'loss': loss_values,
    'val_loss': val_loss_values
})

df_loss.to_csv('loss_curve.csv', index=False)


In [ ]:
# Save the entire model
model.save('my_model.h5')


In [ ]:
from tensorflow.keras.models import load_model

# Load model
model = load_model('my_model.h5')


## Model evaluation

In [ ]:
# Evaluate the model and output the results
y_test_pred = model.predict(test_generator)
y_train_pred = model.predict(train_generator)


In [ ]:
# Calculate R² value and area ratio
from sklearn import metrics
r2_list = []
area_list = []
for i in range(y_test.shape[0]):
    r2_list.append(round(metrics.r2_score(ss.inverse_transform(y_test)[i, :], ss.inverse_transform(y_test_pred)[i, :]), 4))
    area_list.append(np.trapz(ss.inverse_transform(y_test_pred)[i, :], range(200)) / np.trapz(ss.inverse_transform(y_test)[i, :]))

test_r2 = pd.DataFrame({'test_r2': r2_list})['test_r2']
test_r2_mean = test_r2.mean()
test_r2
test_r2_mean

In [ ]:
# Export the R² values to an Excel file
test_r2.to_excel('image_r2_values.xlsx', index=False)

print("R² to image_r2_values.xlsx")

In [ ]:
test_area = pd.DataFrame({'area_list': area_list})['area_list']
test_area_mean = test_area.mean()
test_area
test_area_mean

In [ ]:
# Export the true and predicted values, including image names
y_test_df = pd.DataFrame(ss.inverse_transform(y_test))
y_test_pred_df = pd.DataFrame(ss.inverse_transform(y_test_pred))

y_test_df.insert(0, 'name', test_csv['name'].reset_index(drop=True))
y_test_pred_df.insert(0, 'name', test_csv['name'].reset_index(drop=True))

y_test_df.to_excel('True-test.xlsx', index=False)
y_test_pred_df.to_excel('Pred-test.xlsx', index=False)


In [ ]:
#True vs. predicted comparison plot
import seaborn as sns
for i in range(10):
    plt.figure(figsize=(8,3), dpi=120)
    sns.lineplot(x=range(200),y=ss.inverse_transform(y_test)[i,:],label='True')
    sns.lineplot(x=range(200),y=ss.inverse_transform(y_test_pred)[i,:],label='Preict')
    plt.legend()
    plt.show()


## Interval classification section

In [ ]:
# Custom data generator
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(sparse=False)
encoder = OneHotEncoder(sparse_output=False) 
encoder.fit(labels[['r']])


class CustomDataGenerator2(Sequence):
    def __init__(self, csv_file, directory, batch_size, target_size, label_list, shuffle=True):
        self.csv_file = csv_file
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.label_list = label_list
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.csv_file) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.csv_file.iloc[k] for k in indexes]
        X, y = self.__data_generation(batch)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.csv_file))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch):
        X1 = np.empty((self.batch_size, *self.target_size, 3))
        X2 = np.empty((self.batch_size, 1))  # p
        X3 = np.empty((self.batch_size, 1))  # Lmin
        y = np.empty((self.batch_size, len(encoder.categories_[0])))  

        for i, data in enumerate(batch):
            img_path = f"{self.directory}/{data['name']}"
            image = load_img(img_path, target_size=self.target_size)
            X1[i,] = img_to_array(image) / 255.0
            X2[i,] = data[self.label_list[0]]  # p
            X3[i,] = data[self.label_list[1]]  # Lmin
            y[i,] = encoder.transform([[data[self.label_list[2]]]])[0]

        return [X1, X2, X3], y


In [ ]:
# pip install --upgrade scikit-learn

In [ ]:
label_list2 = ['p', 'Lmin', 'r']
batch_size = 16
target_size = (256, 256)


In [ ]:
train_generator2 = CustomDataGenerator2(train_csv, './images', batch_size, target_size, label_list2, shuffle=False)
test_generator2 = CustomDataGenerator2(test_csv, './images', batch_size, target_size, label_list2, shuffle=False)


In [ ]:
y_train = []
for i in tqdm(range(len(train_generator2))):
    batch_data, batch_labels = train_generator2[i]
    y_train.append(batch_labels)
y_train = np.concatenate(y_train)

y_test = []
for i in tqdm(range(len(test_generator2))):
    batch_data, batch_labels = test_generator2[i]
    y_test.append(batch_labels)
y_test = np.concatenate(y_test)


In [ ]:
y_train.shape,y_test.shape


In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def build_model2(hp):
    input_image = Input(shape=(256, 256, 3))
    input_features1 = Input(shape=(1,))  
    input_features2 = Input(shape=(1,))  
    
    base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    x = base_model(input_image)
    x = Flatten()(x)
    x = Concatenate()([x, input_features1, input_features2])  
    
    for i in range(hp.Int('num_layers', 2, 2)):
        x = Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=32), 
                  activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh']))(x)
        x = Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)

    output = Dense(len(encoder.categories_[0]), activation='softmax')(x)
    model = Model(inputs=[input_image, input_features1, input_features2], outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-4, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.Hyperband(
    build_model2,
    objective='val_accuracy',
    max_epochs=100,
    factor=3,
    directory='my_dir2',
    project_name='helloworld2'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
tuner.search(train_generator2, epochs=15, validation_data=test_generator2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')} 
with the following configuration:
""")
for i in range(best_hps.get('num_layers')):
    print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units, {best_hps.get(f'activation_{i}')} activation, {best_hps.get(f'dropout_{i}')} dropout")

print(f"Optimal learning rate: {best_hps.get('learning_rate')}")


In [ ]:
model2 = tuner.hypermodel.build(best_hps)
history2 = model2.fit(
    train_generator2,
    steps_per_epoch=len(train_generator2),
    epochs=100,
    validation_data=test_generator2,
    validation_steps=len(test_generator2),
    callbacks=[tf.keras.callbacks.ModelCheckpoint("./best_model2.h5", save_best_only=True, monitor='val_accuracy')]
)


In [ ]:
def plot_model_history2(model_history):
    fig, axs = plt.subplots(1, 2, figsize=(10, 3), dpi=120)
    axs[0].plot(range(1, len(model_history.history['accuracy']) + 1), model_history.history['accuracy'])
    axs[0].plot(range(1, len(model_history.history['val_accuracy']) + 1), model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['train', 'val'], loc='best')
    
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
plot_model_history2(history2)


In [ ]:
accuracy_values = history2.history['accuracy'] 
val_accuracy_values = history2.history['val_accuracy']  
loss_values = history2.history['loss']  
val_loss_values = history2.history['val_loss'] 

num_epoch = len(accuracy_values)

df_metrics = pd.DataFrame({
    'epoch': np.arange(1, num_epoch + 1),
    'accuracy': accuracy_values,
    'val_accuracy': val_accuracy_values,
    'loss': loss_values,
    'val_loss': val_loss_values
})

df_metrics.to_csv('classification_training_curve.csv', index=False)

print(" classification_training_curve.csv")


In [ ]:
model2.save('my_model2.h5')


In [ ]:
from tensorflow.keras.models import load_model

model2 = tf.keras.models.load_model('my_model2.h5')


## Model evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score ,roc_curve, auc,confusion_matrix ,accuracy_score,roc_auc_score,auc,brier_score_loss
from sklearn import metrics
def try_different_method(y_train,y_pred_train1,y_test,y_pred_test1):
    print('Train:')
    precision = precision_score(y_train,y_pred_train1,average='macro')
    recall = recall_score(y_train,y_pred_train1,average='macro')
    f1score = f1_score(y_train, y_pred_train1,average='macro')
    accuracy=accuracy_score(y_train, y_pred_train1)
    print("ACC: ", '%.4f'%float(accuracy),"F1：", '%.4f'%float(f1score),"Precision:", '%.4f'%float(precision),\
    "Recall:   ",'%.4f'%float(recall))
    train_metrics=[accuracy,precision,recall,f1score]
    print('Model Train Report: \n',metrics.classification_report(y_train,y_pred_train1,digits=4))
    print('*'*50)
    print('Test:')

    precision = precision_score(y_test,y_pred_test1,average='macro')
    recall = recall_score(y_test,y_pred_test1,average='macro')
    f1score = f1_score(y_test, y_pred_test1,average='macro')
    accuracy=accuracy_score(y_test, y_pred_test1)
    print("ACC: ", '%.4f'%float(accuracy),"F1：", '%.4f'%float(f1score),"Precision:", '%.4f'%float(precision),\
    "Recall:   ",'%.4f'%float(recall))
    print('Model Test Report: \n',metrics.classification_report(y_test,y_pred_test1,digits=4))
    test_metrics=[accuracy,precision,recall,f1score]
    return train_metrics,test_metrics
import matplotlib.pyplot as plt 
import itertools
def plot_confusion_matrix(cm, classes,title='Confusion matrix',cmap=plt.cm.Blues):
    # plt.figure(figsize=(12,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center",color="#00FFFF" if cm[i, j] > thresh else "red")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
# Evaluate the classification model and output the results
y_test_pred = model2.predict(test_generator2)
y_train_pred = model2.predict(train_generator2)


In [ ]:
labels['r'].unique()


In [ ]:
train_metrics,test_metrics=try_different_method(np.argmax(y_train,axis=1),np.argmax(y_train_pred,axis=1),
                                                np.argmax(y_test,axis=1),np.argmax(y_test_pred,axis=1))


In [ ]:
y_test_df = pd.DataFrame(np.argmax(y_test,axis=1)+3)
y_test_pred_df = pd.DataFrame(np.argmax(y_test_pred,axis=1)+3)

y_test_df.insert(0, 'name', test_csv['name'].reset_index(drop=True))
y_test_pred_df.insert(0, 'name', test_csv['name'].reset_index(drop=True))

y_test_df.to_excel('True-test2.xlsx', index=False)
y_test_pred_df.to_excel('Pred-test2.xlsx', index=False)


In [ ]:
plt.figure(figsize=(10,6), dpi=120)
plt.subplot(2, 2, 1)
#train
cnf_matrix=metrics.confusion_matrix(np.argmax(y_train,axis=1),np.argmax(y_train_pred,axis=1))
plot_confusion_matrix(cnf_matrix,[3,4,5,6,7],title='Train',cmap=plt.cm.Blues)
#test
plt.subplot(2, 2, 2)
cnf_matrix=metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_test_pred,axis=1))
plot_confusion_matrix(cnf_matrix,[3,4,5,6,7],title='Test',cmap=plt.cm.Blues)
plt.tight_layout()
plt.savefig('confusion_matrix.jpg',dpi=300,bbox_inches = 'tight')
plt.show()


## Model prediction

In [ ]:
# Load model
model1 = load_model('my_model.h5')  # Load curve prediction model
model2 = load_model('my_model2.h5')  # Load r prediction model


In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('my_model.h5')
model2 = load_model('my_model2.h5')

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
ss = joblib.load('ss.pkl')
os.makedirs('./predicted_curves', exist_ok=True) 

predictions = []
image_names = []

# Traverse images in the new folder
new_image_folder = './val_images/'  # New image folder path
for img_name in os.listdir(new_image_folder):
    # Read image
    img_path = os.path.join(new_image_folder, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))

    # Calculate porosity
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
    white_pixels = np.sum(gray_img > 80)  
    total_pixels = gray_img.size  # 
    p_value = np.array([[white_pixels / total_pixels]]) 
    
    # Calculate Lmin
    height, width = gray_img.shape
    min_black_count = float('inf')
    for y in range(height):
        row = gray_img[y, :]
        is_black = (row < 127)
        black_count = is_black.sum()
        if black_count < min_black_count:
            min_black_count = black_count
    Lmin_value = np.array([[min_black_count]]) 

    img = np.reshape(img, [1, 256, 256, 3]) / 255.0

    # Call model2 to predict the interval, passing all three inputs
    r_value = np.argmax(model2.predict([img, p_value, Lmin_value]), axis=1)[0] + 3

    # Predict the curve by passing the image, p, Lmin, and r
    prediction = model1.predict([img, p_value, Lmin_value, np.array([[r_value]])])

    
    inverse_prediction = ss.inverse_transform(prediction)[0]

 
    predictions.append(inverse_prediction)
    image_names.append(img_name)


    plt.figure(figsize=(10, 4))
    plt.plot(range(200), inverse_prediction)  
    plt.title(f'Predicted Curve for {img_name}')
    plt.xlabel('Data Points')
    plt.ylabel('Predicted Value')
    plt.savefig(f'./predicted_curves/{img_name}_curve.png')  
    plt.close()

# 导出所有预测数据到Excel
predictions_df = pd.DataFrame(predictions)
predictions_df.insert(0, 'Image Name', image_names)  
predictions_df.to_excel('predicted_curves_data.xlsx', index=False)  
